In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [2]:
!pip install pytorch-tabnet

In [3]:
%load_ext tensorboard

# Load data and data preprocessing

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import sys
import os
path = '/content/drive/My Drive'
sys.path.append(path)
os.chdir(path)
!ls

 add_labels.ipynb		 jsondemo.json
 aliyun_coding.ipynb		 kdd_pre_final.csv
'Anyfile Notepad Files'		 kmcuda-master
'Anyfile Notepad Files (1)'	'kmeans_scipy&sklearn.ipynb'
 assignment4.ipynb		 kmeans_test.ipynb
 cache				 matrix_all.mat
'Colab Notebooks'		 mchar_train
 cosine_metric_learning-master	 mchar_train.json
'deep computer vision lab'	 mchar_val.json
 deep-person-reid		 modelDIY
 deep_sort-master		 model_stacking_final_assignment.ipynb
 deep_sort_pytorch-origin	 mydata.json
 demo.bin			 test_final.csv
 DENet_GlaucomaScreen-master	 train_final.csv
 dt_output			 waymo-open-dataset-master
 frames				 yeast.txt
 geeks14.png			 代码
'graduation project'


In [6]:
seed = 42 # for the same data division

kf = KFold(n_splits=5, random_state=seed,shuffle=True)
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')

X_train = df_train.drop(columns=['loan_status']).values
Y_train = df_train['loan_status'].values.astype(int)
X_test = df_test.drop(columns=['loan_status']).values
Y_test = df_test['loan_status'].values.astype(int)

In [7]:
X_train.shape, Y_train.shape

((50000, 145), (50000,))

In [8]:
# split data for five fold

five_fold_data = []

for train_index, eval_index in kf.split(X_train):
    x_train, x_eval = X_train[train_index], X_train[eval_index]
    y_train, y_eval = Y_train[train_index], Y_train[eval_index]
    
    five_fold_data.append([(x_train, y_train), (x_eval, y_eval)])

# Algorithm

In [61]:
def get_model(param):
    model_list = []
    for idx, [(x_train, y_train), (x_eval, y_eval)] in enumerate(five_fold_data):
        print('{}-th model is training:'.format(idx))
        train_data = lgb.Dataset(x_train, label=y_train)
        validation_data = lgb.Dataset(x_eval, label=y_eval)
        bst = lgb.train(param, train_data, valid_sets=[validation_data])
        model_list.append(bst)
    return model_list

# train

In [62]:
param_base = {'num_leaves': 31, 'objective': 'binary', 'metric': 'binary', 'num_round':1000}

param_fine_tuning = {'num_thread': 8,'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000, 
                     'learning_rate': 3e-3, 'feature_fraction': 0.6, 'bagging_fraction': 0.8}

In [63]:

# base param train
param_base_model = get_model(param_base)

# param fine tuning
%time param_fine_tuning_model = get_model(param_fine_tuning)

0-th model is training:


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


流式输出内容被截断，只能显示最后 5000 行内容。
[6]	valid_0's binary_logloss: 0.307685
[7]	valid_0's binary_logloss: 0.292626
[8]	valid_0's binary_logloss: 0.279943
[9]	valid_0's binary_logloss: 0.269078
[10]	valid_0's binary_logloss: 0.259697
[11]	valid_0's binary_logloss: 0.251741
[12]	valid_0's binary_logloss: 0.244823
[13]	valid_0's binary_logloss: 0.238844
[14]	valid_0's binary_logloss: 0.233637
[15]	valid_0's binary_logloss: 0.229147
[16]	valid_0's binary_logloss: 0.225266
[17]	valid_0's binary_logloss: 0.221848
[18]	valid_0's binary_logloss: 0.218858
[19]	valid_0's binary_logloss: 0.216144
[20]	valid_0's binary_logloss: 0.213877
[21]	valid_0's binary_logloss: 0.211905
[22]	valid_0's binary_logloss: 0.210164
[23]	valid_0's binary_logloss: 0.208786
[24]	valid_0's binary_logloss: 0.20746
[25]	valid_0's binary_logloss: 0.206263
[26]	valid_0's binary_logloss: 0.205241
[27]	valid_0's binary_logloss: 0.204314
[28]	valid_0's binary_logloss: 0.203611
[29]	valid_0's binary_logloss: 0.202913
[30]	valid_0's bin

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


流式输出内容被截断，只能显示最后 5000 行内容。
[7]	valid_0's binary_logloss: 0.495869
[8]	valid_0's binary_logloss: 0.494066
[9]	valid_0's binary_logloss: 0.492281
[10]	valid_0's binary_logloss: 0.491962
[11]	valid_0's binary_logloss: 0.490227
[12]	valid_0's binary_logloss: 0.488481
[13]	valid_0's binary_logloss: 0.486748
[14]	valid_0's binary_logloss: 0.485055
[15]	valid_0's binary_logloss: 0.484736
[16]	valid_0's binary_logloss: 0.483075
[17]	valid_0's binary_logloss: 0.481424
[18]	valid_0's binary_logloss: 0.479774
[19]	valid_0's binary_logloss: 0.478135
[20]	valid_0's binary_logloss: 0.477829
[21]	valid_0's binary_logloss: 0.476223
[22]	valid_0's binary_logloss: 0.474614
[23]	valid_0's binary_logloss: 0.473031
[24]	valid_0's binary_logloss: 0.471449
[25]	valid_0's binary_logloss: 0.469909
[26]	valid_0's binary_logloss: 0.468362
[27]	valid_0's binary_logloss: 0.466824
[28]	valid_0's binary_logloss: 0.46654
[29]	valid_0's binary_logloss: 0.466307
[30]	valid_0's binary_logloss: 0.464816
[31]	valid_0's bi

# test

In [11]:
def test_model(model_list):
    data = X_test
    five_fold_pred = np.zeros((5, len(X_test)))
    for i, bst in enumerate(model_list):
        ypred = bst.predict(data, num_iteration=bst.best_iteration)
        five_fold_pred[i] = ypred
    ypred_mean = (five_fold_pred.mean(axis=-2)>0.5).astype(int)
    return accuracy_score(ypred_mean, Y_test)

In [13]:
base_score = test_model(param_base_model)
%time fine_tuning_score = test_model(param_fine_tuning_model)

print('base: {}, fine tuning: {}'.format(base_score, fine_tuning_score))

CPU times: user 35.6 s, sys: 0 ns, total: 35.6 s
Wall time: 18.1 s
base: 0.91584, fine tuning: 0.91762


In [64]:
def test_model_raw(model_list):
    data = X_test
    five_fold_pred = np.zeros((5, len(X_test)))
    for i, bst in enumerate(model_list):
        ypred = bst.predict(data, num_iteration=bst.best_iteration)
        five_fold_pred[i] = ypred
    return (five_fold_pred.mean(axis=-2)>0.5).astype(int)

# TabNet

In [14]:
# data filling
from sklearn.impute import SimpleImputer
import numpy as np

seed = 42 # for the same data division

kf = KFold(n_splits=5, random_state=seed,shuffle=True)
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')

X_train = df_train.drop(columns=['loan_status']).values
Y_train = df_train['loan_status'].values.astype(int)
X_test = df_test.drop(columns=['loan_status']).values
Y_test = df_test['loan_status'].values.astype(int)

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

five_fold_data = []

for train_index, eval_index in kf.split(X_train):
    x_train, x_eval = X_train[train_index], X_train[eval_index]
    y_train, y_eval = Y_train[train_index], Y_train[eval_index]
    
    five_fold_data.append([(x_train, y_train), (x_eval, y_eval)])

In [112]:
%%time
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error


# TabNetPretrainer
# This is not right?
# unsupervised_model = TabNetPretrainer(
#     optimizer_fn=torch.optim.Adam,
#     optimizer_params=dict(lr=2e-2),
#     mask_type='entmax' # "sparsemax"
# )

# unsupervised_model.fit(
#     X_train=X_train,
#     eval_set=[X_train],
#     pretraining_ratio=0.8,
# )

clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax' # This will be overwritten if using pretrain model
)

model_sets = []
for [(x_train, y_train), (x_eval, y_eval)] in five_fold_data:
  clf.fit(
      X_train=x_train, y_train=y_train,
      eval_set=[(x_train, y_train), (x_eval, y_eval)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss'],
      from_unsupervised=None
  )
  model_sets.append(clf)

def tabnet_test(model_sets):
    results = np.zeros((5, len(X_test)))
    for i, model in enumerate(model_sets):
        preds = model.predict(X_test)
        results[i] = preds
    return (results.mean(axis=-2)>0.5).astype(int)
print(accuracy_score(y_pred=tabnet_test(model_sets), y_true=Y_test))
# cost 3min 45sec

Device used : cuda
epoch 0  | loss: 0.38702 | train_logloss: 1.67905 | valid_logloss: 1.70978 |  0:00:02s
epoch 1  | loss: 0.227   | train_logloss: 1.3579  | valid_logloss: 1.38082 |  0:00:04s
epoch 2  | loss: 0.21386 | train_logloss: 1.22835 | valid_logloss: 1.24949 |  0:00:07s
epoch 3  | loss: 0.21019 | train_logloss: 0.81221 | valid_logloss: 0.82624 |  0:00:09s
epoch 4  | loss: 0.20906 | train_logloss: 0.45913 | valid_logloss: 0.46521 |  0:00:12s
epoch 5  | loss: 0.20781 | train_logloss: 0.30766 | valid_logloss: 0.30784 |  0:00:14s
epoch 6  | loss: 0.20717 | train_logloss: 0.25215 | valid_logloss: 0.25186 |  0:00:17s
epoch 7  | loss: 0.20522 | train_logloss: 0.24433 | valid_logloss: 0.24316 |  0:00:19s
epoch 8  | loss: 0.20686 | train_logloss: 0.21504 | valid_logloss: 0.21657 |  0:00:22s
epoch 9  | loss: 0.20609 | train_logloss: 0.20133 | valid_logloss: 0.20433 |  0:00:24s
epoch 10 | loss: 0.20228 | train_logloss: 0.19831 | valid_logloss: 0.20075 |  0:00:27s
epoch 11 | loss: 0.20253

DeepTables

In [74]:
!pip install deeptables

     |████████████████████████████████| 2.2MB 9.8MB/s 
     |████████████████████████████████| 4.3MB 42.9MB/s 
     |████████████████████████████████| 394.7MB 37kB/s 
     |████████████████████████████████| 112kB 49.8MB/s 
     |████████████████████████████████| 2.0MB 35.8MB/s 
     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 102kB 7.7MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 112kB 37.6MB/s 
     |████████████████████████████████| 286kB 30.1MB/s 
  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp37-none-any.whl size=18178 sha256=84e10ecfc68cc2e92694212526bb58cf168e4815e84dc363069b665cd8785125
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled table

In [75]:
# Reference: https://deeptables.readthedocs.io/en/latest/examples.html
%%time
from deeptables.models import deeptable, deepnets

config = deeptable.ModelConfig(nets=deepnets.DeepFM)
dt = deeptable.DeepTable(config=config)
model_sets = []
for [(x_train, y_train), (x_eval, y_eval)] in five_fold_data:
  dt.fit(x_train, y_train, epochs=20)
  # result = dt.evaluate(x_eval, y_eval, batch_size=512, verbose=0)
  model_sets.append(dt)

for i, model in enumerate(model_sets):
  results = np.zeros((5, len(X_test)))
  for i, model in enumerate(model_sets):
      preds = model.predict(X_test)
      results[i] = preds
  preds = (results.mean(axis=-2)>0.5).astype(int)
print(accuracy_score(preds, Y_test))

Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)


2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features taken 0.26346778869628906s
Imputation taken 0.2361011505126953s
Categorical encoding taken 3.0040740966796875e-05s
Injected a callback [EarlyStopping]. monitor:val_accuracy, patience:1, mode:max
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['input_continuous_all: (142)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
None
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 142)
---------------------------------------------------------
nets: ['fm_nets', 'dnn_nets', 'linear']
---------------------------------------------------------
fm: input_

Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)


Model has been saved to:dt_output/dt_20210505 151545_fm_nets_dnn_nets_linear/fm_nets+dnn_nets+linear.h5
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features taken 0.3057253360748291s
Imputation taken 0.2598907947540283s
Categorical encoding taken 3.337860107421875e-05s
Injected a callback [EarlyStopping]. monitor:val_accuracy, patience:1, mode:max
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['input_continuous_all: (143)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
None
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 143)
---------------------------------------------------------
nets: 

Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)


...

Epoch 00004: early stopping
Model has been saved to:dt_output/dt_20210505 151545_fm_nets_dnn_nets_linear/fm_nets+dnn_nets+linear.h5
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features taken 0.2518584728240967s
Imputation taken 0.2239978313446045s
Categorical encoding taken 2.86102294921875e-05s
Injected a callback [EarlyStopping]. monitor:val_accuracy, patience:1, mode:max
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['input_continuous_all: (143)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
None
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 143)
-------------------------------------

Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)


Model has been saved to:dt_output/dt_20210505 151545_fm_nets_dnn_nets_linear/fm_nets+dnn_nets+linear.h5


Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)
Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)
Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',
       'x_143', 'x_144'],
      dtype='object', length=145)
Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       ...
       'x_135', 'x_136', 'x_137', 'x_138', 'x_139', 'x_140', 'x_141', 'x_142',


0.89364
CPU times: user 45.9 s, sys: 3.06 s, total: 49 s
Wall time: 41.8 s


# CatBoost

In [55]:
!pip install catboost

     |████████████████████████████████| 67.3MB 53kB/s 


In [71]:
from catboost import CatBoostClassifier, Pool

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=10)
model_sets = []
for [(x_train, y_train), (x_eval, y_eval)] in five_fold_data:
    eval_set = Pool(x_eval, label=y_eval)
    model.fit(x_train, y_train, eval_set=[eval_set], use_best_model=True, early_stopping_rounds=10)
    model_sets.append(model)

results = np.zeros((5, len(X_test)))
for i, model in enumerate(model_sets):
    preds = model.predict(X_test)
    results[i] = preds
results = (results.mean(axis=-2)>0.5).astype(int)
print(accuracy_score(results, Y_test))

0:	learn: 0.5680362	test: 0.5688249	best: 0.5688249 (0)	total: 26.6ms	remaining: 2.63s
1:	learn: 0.4648060	test: 0.4663274	best: 0.4663274 (1)	total: 111ms	remaining: 5.45s
2:	learn: 0.3946790	test: 0.3966461	best: 0.3966461 (2)	total: 196ms	remaining: 6.33s
3:	learn: 0.3425630	test: 0.3449286	best: 0.3449286 (3)	total: 287ms	remaining: 6.88s
4:	learn: 0.3029116	test: 0.3060841	best: 0.3060841 (4)	total: 374ms	remaining: 7.11s
5:	learn: 0.2789660	test: 0.2824040	best: 0.2824040 (5)	total: 463ms	remaining: 7.25s
6:	learn: 0.2600535	test: 0.2634463	best: 0.2634463 (6)	total: 555ms	remaining: 7.38s
7:	learn: 0.2451501	test: 0.2489839	best: 0.2489839 (7)	total: 643ms	remaining: 7.39s
8:	learn: 0.2320690	test: 0.2369122	best: 0.2369122 (8)	total: 728ms	remaining: 7.37s
9:	learn: 0.2240924	test: 0.2297633	best: 0.2297633 (9)	total: 816ms	remaining: 7.34s
10:	learn: 0.2160827	test: 0.2224681	best: 0.2224681 (10)	total: 904ms	remaining: 7.32s
11:	learn: 0.2127858	test: 0.2193193	best: 0.219319

# Voting Classifier (Hard)

In [72]:
%%time
import random
# ML_results = tabnet_test(model_sets)
# DL_results = preds
lgb_results = test_model_raw(param_fine_tuning_model)
Catboost_results = test_prediction
# if the prediction results are the same, then trust the results
# else use a random function
ultimate_results = np.zeros((len(X_test),1))
for i in range(len(X_test)):
    if Catboost_results[i] == lgb_results[i]:
        ultimate_results[i] = lgb_results[i]
    else:
        ultimate_results[i] = Catboost_results[i] + lgb_results[i] # (ML_results[i] + DL_results[i] + lgb_results[i]) // 2 or random.choice([ML_results[i], DL_results[i], lgb_results[i]])
print(accuracy_score(ultimate_results, Y_test))

0.91762
CPU times: user 39 s, sys: 10.9 ms, total: 39 s
Wall time: 20 s


# Model Stacking

Lightgbm, XGBoost, TabNet, DeepTable

In [25]:
import numpy as np
from sklearn.model_selection import KFold

class BasicModel(object):
    """Parent class of basic models"""
    def train(self, x_train, y_train, x_val, y_val):
        """return a trained model and eval metric of validation data"""
        pass
    
    def predict(self, model, x_test):
        """return the predicted result of test data"""
        pass
    
    def get_oof(self, x_train, y_train, x_test, n_folds = 5):
        """K-fold stacking"""
        num_train, num_test = x_train.shape[0], x_test.shape[0]
        oof_train = np.zeros((num_train,)) 
        oof_test = np.zeros((num_test,))
        oof_test_all_fold = np.zeros((num_test, n_folds))
        aucs = []
        KF = KFold(n_splits = n_folds, random_state=2017)
        for i, (train_index, val_index) in enumerate(KF.split(x_train)):
            print('{0} fold, train {1}, val {2}'.format(i, 
            len(train_index),
            len(val_index)))
            x_tra, y_tra = x_train[train_index], y_train[train_index]
            x_val, y_val = x_train[val_index], y_train[val_index]
            model, auc = self.train(x_tra, y_tra, x_val, y_val)
            aucs.append(auc)
            oof_train[val_index] = self.predict(model, x_val)
            oof_test_all_fold[:, i] = self.predict(model, x_test)
        oof_test = np.mean(oof_test_all_fold, axis=1)
        print('all aucs/logloss {0}, average {1}'.format(aucs, np.mean(aucs)))
        return oof_train, oof_test

import lightgbm as lgb
class LGBClassifier(BasicModel):
    def __init__(self):
        self.num_boost_round = 1200
        self.early_stopping_rounds = 15
        # self.params = {
        #     'task': 'train',
        #     'boosting_type': 'gbdt',
        #     'objective': 'binary',
        #     'metric': {'auc', 'binary_logloss'},
        #     'num_leaves': 80,
        #     'learning_rate': 0.02,
        #     # 'scale_pos_weight': 1.5,
        #     'feature_fraction': 0.6,
        #     'bagging_fraction': 0.8,
        #     'bagging_freq': 5,
        #     'max_bin': 300,
        #     'is_unbalance': True,
        #     'lambda_l2': 5.0,
        #     'verbose' : -1
        #     }
        self.params = {'num_thread': 8,'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': self.num_boost_round, 
                      'learning_rate': 3e-3, 'feature_fraction': 0.6, 'bagging_fraction': 0.8, 'metric': {'binary_logloss'}}
        
    def train(self, x_train, y_train, x_val, y_val):
        print('train with lgb model')
        lgbtrain = lgb.Dataset(x_train, y_train)
        lgbval = lgb.Dataset(x_val, y_val)
        model = lgb.train(self.params, 
                  lgbtrain,
                  valid_sets = [lgbval],
                  num_boost_round = self.num_boost_round,
                  early_stopping_rounds = self.early_stopping_rounds)
        return model, model.best_score['valid_0']['binary_logloss']
    
    def predict(self, model, x_test):
        print('test with lgb model')
        return model.predict(x_test, num_iteration=model.best_iteration)



lgb_classifier = LGBClassifier()
lgb_oof_train, lgb_oof_test = lgb_classifier.get_oof(X_train, Y_train, X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


流式输出内容被截断，只能显示最后 5000 行内容。
[1034]	valid_0's binary_logloss: 0.190565
[1035]	valid_0's binary_logloss: 0.190534
[1036]	valid_0's binary_logloss: 0.190501
[1037]	valid_0's binary_logloss: 0.190469
[1038]	valid_0's binary_logloss: 0.190434
[1039]	valid_0's binary_logloss: 0.190402
[1040]	valid_0's binary_logloss: 0.190373
[1041]	valid_0's binary_logloss: 0.190336
[1042]	valid_0's binary_logloss: 0.190307
[1043]	valid_0's binary_logloss: 0.190274
[1044]	valid_0's binary_logloss: 0.190273
[1045]	valid_0's binary_logloss: 0.19024
[1046]	valid_0's binary_logloss: 0.190211
[1047]	valid_0's binary_logloss: 0.190178
[1048]	valid_0's binary_logloss: 0.190148
[1049]	valid_0's binary_logloss: 0.190119
[1050]	valid_0's binary_logloss: 0.190086
[1051]	valid_0's binary_logloss: 0.190056
[1052]	valid_0's binary_logloss: 0.190022
[1053]	valid_0's binary_logloss: 0.189994
[1054]	valid_0's binary_logloss: 0.189961
[1055]	valid_0's binary_logloss: 0.189933
[1056]	valid_0's binary_logloss: 0.189911
[1057]	v

In [24]:
import xgboost as xgb
class XGBClassifier(BasicModel):
    def __init__(self):
        """set parameters"""
        self.num_rounds=1000
        self.early_stopping_rounds = 15
        self.params = {
            'objective': 'binary:logistic',
            'eta': 0.1,
            'max_depth': 10,
            'eval_metric': 'logloss',
            'seed': 0,
            'silent' : 0
         }
        
    def train(self, x_train, y_train, x_val, y_val):
        print('train with xgb model')
        xgbtrain = xgb.DMatrix(x_train, y_train)
        xgbval = xgb.DMatrix(x_val, y_val)
        watchlist = [(xgbtrain,'train'), (xgbval, 'val')]
        model = xgb.train(self.params, xgbtrain, self.num_rounds,
                  watchlist, early_stopping_rounds = self.early_stopping_rounds)
        return model, float(model.eval(xgbval).split()[1].split(':')[1])

    def predict(self, model, x_test):
        print('test with xgb model')
        xgbtest = xgb.DMatrix(x_test)
        return model.predict(xgbtest)

xgb_classifier = XGBClassifier()
xgb_oof_train, xgb_oof_test = xgb_classifier.get_oof(X_train, Y_train, X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0 fold, train 40000, val 10000
train with xgb model
[0]	train-logloss:0.61809	val-logloss:0.619926
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 15 rounds.
[1]	train-logloss:0.556437	val-logloss:0.560094
[2]	train-logloss:0.504824	val-logloss:0.509934
[3]	train-logloss:0.460993	val-logloss:0.467965
[4]	train-logloss:0.42334	val-logloss:0.432243
[5]	train-logloss:0.390931	val-logloss:0.401515
[6]	train-logloss:0.362538	val-logloss:0.375109
[7]	train-logloss:0.337698	val-logloss:0.352197
[8]	train-logloss:0.315991	val-logloss:0.332118
[9]	train-logloss:0.296903	val-logloss:0.314775
[10]	train-logloss:0.280029	val-logloss:0.299678
[11]	train-logloss:0.264975	val-logloss:0.2863
[12]	train-logloss:0.251267	val-logloss:0.274528
[13]	train-logloss:0.239117	val-logloss:0.264218
[14]	train-logloss:0.228248	val-logloss:0.254728
[15]	train-logloss:0.218361	val-logloss:0.246643
[16]	train-logloss:0.209632	val

In [17]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

class tabnetClassifier(BasicModel):
    def __init__(self):
        self.clf = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                      optimizer_params=dict(lr=2e-2),
                      scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
                      scheduler_fn=torch.optim.lr_scheduler.StepLR,
                      mask_type='entmax' # This will be overwritten if using pretrain model
)
    
    def train(self, x_train, y_train, x_val, y_val):
        print('train with tabnet model')
        self.clf.fit(
      X_train=x_train, y_train=y_train,
      eval_set=[(x_train, y_train), (x_val, y_val)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss'],
      from_unsupervised=None
  )
        return self.clf, 0
    
    def predict(self, model, x_test):
        print('test with tabnet model')
        return model.predict(x_test)

tabnet_classifier = tabnetClassifier()
tabnet_oof_train, tabnet_oof_test = tabnet_classifier.get_oof(X_train, Y_train, X_test)

Device used : cpu
0 fold, train 40000, val 10000
train with tabnet model


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


epoch 0  | loss: 0.38743 | train_logloss: 2.20595 | valid_logloss: 2.05434 |  0:00:08s
epoch 1  | loss: 0.22769 | train_logloss: 1.30343 | valid_logloss: 1.22936 |  0:00:17s
epoch 2  | loss: 0.21648 | train_logloss: 1.08077 | valid_logloss: 1.01063 |  0:00:27s
epoch 3  | loss: 0.21443 | train_logloss: 0.71607 | valid_logloss: 0.67037 |  0:00:36s
epoch 4  | loss: 0.21086 | train_logloss: 0.4822  | valid_logloss: 0.45421 |  0:00:45s
epoch 5  | loss: 0.21084 | train_logloss: 0.29617 | valid_logloss: 0.27574 |  0:00:54s
epoch 6  | loss: 0.21042 | train_logloss: 0.25416 | valid_logloss: 0.23376 |  0:01:03s
epoch 7  | loss: 0.20949 | train_logloss: 0.22468 | valid_logloss: 0.20501 |  0:01:12s
epoch 8  | loss: 0.21157 | train_logloss: 0.21423 | valid_logloss: 0.19418 |  0:01:21s
epoch 9  | loss: 0.2077  | train_logloss: 0.20874 | valid_logloss: 0.19112 |  0:01:30s
epoch 10 | loss: 0.20654 | train_logloss: 0.20154 | valid_logloss: 0.18434 |  0:01:39s
epoch 11 | loss: 0.20834 | train_logloss: 0

In [88]:
from catboost import CatBoostClassifier

class catboostclassifier(BasicModel):
    def __init__(self):
        self.clf = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=10)
    
    def train(self, x_train, y_train, x_val, y_val):
        print('train with catboost model')
        eval_set = Pool(x_val, label=y_val)
        self.clf.fit(x_train, y_train, eval_set=[eval_set], use_best_model=True, early_stopping_rounds=10)
        return self.clf, 0
    
    def predict(self, model, x_test):
        print('test with catboost model')
        return model.predict(x_test)

catboost_classifier = catboostclassifier()
catboost_oof_train, catboost_oof_test = catboost_classifier.get_oof(X_train, Y_train, X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0 fold, train 40000, val 10000
train with catboost model
0:	learn: 0.5506747	test: 0.5476365	best: 0.5476365 (0)	total: 82ms	remaining: 8.11s
1:	learn: 0.4552314	test: 0.4502181	best: 0.4502181 (1)	total: 163ms	remaining: 7.97s
2:	learn: 0.3963771	test: 0.3901282	best: 0.3901282 (2)	total: 243ms	remaining: 7.85s
3:	learn: 0.3440659	test: 0.3369362	best: 0.3369362 (3)	total: 325ms	remaining: 7.8s
4:	learn: 0.3083050	test: 0.3001606	best: 0.3001606 (4)	total: 407ms	remaining: 7.72s
5:	learn: 0.2821457	test: 0.2736141	best: 0.2736141 (5)	total: 491ms	remaining: 7.69s
6:	learn: 0.2642022	test: 0.2551273	best: 0.2551273 (6)	total: 573ms	remaining: 7.61s
7:	learn: 0.2493781	test: 0.2396306	best: 0.2396306 (7)	total: 652ms	remaining: 7.5s
8:	learn: 0.2387520	test: 0.2287633	best: 0.2287633 (8)	total: 733ms	remaining: 7.41s
9:	learn: 0.2302455	test: 0.2209988	best: 0.2209988 (9)	total: 821ms	remaining: 7.39s
10:	learn: 0.2222660	test: 0.2128599	best: 0.2128599 (10)	total: 903ms	remaining: 7.3s

In [98]:
input_train = [lgb_oof_train, catboost_oof_train] 
input_test = [lgb_oof_test, catboost_oof_test]

stacked_train = np.concatenate([f.reshape(-1, 1) for f in input_train], axis=1)
stacked_test = np.concatenate([f.reshape(-1, 1) for f in input_test], axis=1)

In [99]:
from sklearn.linear_model import LogisticRegression

final_model = LogisticRegression()
final_model.fit(stacked_train, Y_train)
test_prediction = final_model.predict(stacked_test)

print(accuracy_score(test_prediction, Y_test))

0.91584


In [100]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators = 100,random_state = 20)
rnd_clf.fit(stacked_train, Y_train)
test_prediction = rnd_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.8831


In [101]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(stacked_train, Y_train)
test_prediction = svc_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.91816


In [102]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier()
ada_clf.fit(stacked_train,Y_train)
test_prediction = ada_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.91422


In [103]:
from sklearn.naive_bayes import GaussianNB
nby_clf = GaussianNB()
nby_clf.fit(stacked_train,Y_train)
test_prediction = nby_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.91772


In [104]:
from sklearn.neighbors import KNeighborsClassifier
knc_clf = KNeighborsClassifier()
knc_clf.fit(stacked_train,Y_train)
test_prediction = knc_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.90792


In [105]:
# voting classifier
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[
    ('log_clf', LogisticRegression()),
    ('svm_clf', SVC()),
    ('gnb_clf', GaussianNB())
], voting='hard')

voting_clf.fit(stacked_train, Y_train)
test_prediction = voting_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.91802


In [106]:
# soft voting classifier
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[
    ('log_clf', LogisticRegression()),
    ('svm_clf', SVC(probability=True)),
    ('gnb_clf', GaussianNB())
], voting='soft')

voting_clf.fit(stacked_train, Y_train)
test_prediction = voting_clf.predict(stacked_test)
print(accuracy_score(test_prediction, Y_test))

0.91826


# **Model Blending**

1. Create a holdout of 10% of the train set

2. Fit a first stage model on the train set without the holdout

3. Predict the untouched holdout with the trained first stage model

4. Split the predicted holdout in k folds

5. Fit a second stage model on k-1 folds and predict the kth fold
Repeat 5) to predict each fold

6. The CV error of the second stage is calculated on each predicted fold

In [108]:
import time
from numpy import hstack
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore")

# get a list of base models
def get_models():
    models = list()
    models.append(('lr', LogisticRegression(max_iter=500)))
    models.append(('cart', DecisionTreeClassifier()))
    models.append(('bayes', GaussianNB()))
    # models.append(("svm", SVC(probability=True)))
    return models


# fit the blending ensemble
def fit_ensemble(models, X_train, X_val, y_train, y_val):
    # fit all models on the training set and predict on hold out set
    meta_X = list()
    for _, model in models: 
        model.fit(X_train, y_train)
        yhat = model.predict_proba(X_val)
        meta_X.append(yhat)
    meta_X = hstack(meta_X)
    # params = {'num_thread': 8,'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000,
    #                  'learning_rate': 3e-3, 'feature_fraction': 0.6, 'bagging_fraction': 0.8}
    # blender = lgb.LGBMClassification(**params)
    # blender.fit(meta_X, y_val)
    # blender = LogisticRegressionCV(cv=5, random_state=0)
    blender = SVC(probability=False)
    # blender = GaussianNB()
    # blender = AdaBoostClassifier()
    blender.fit(meta_X, y_val)
    return blender


def predict_ensemble(models, blender, X_test):
    meta_X = list()
    for name, model in models:
        yhat = model.predict_proba(X_test)
        meta_X.append(yhat)
    meta_X = hstack(meta_X)
    return blender.predict(meta_X)


X, y = X_train, Y_train
X_train_1, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1)
# print('Train: %s, Val: %s, Test: %s' % (X_train.shape, X_val.shape, X_test.shape))
start = time.time()
models = get_models()
print("Get model finished.")
blender = fit_ensemble(models, X_train_1, X_val, y_train, y_val)
print("Train model finished.")
yhat = predict_ensemble(models, blender, X_test)
print("Test model finished.")
score = accuracy_score(Y_test, yhat)
print('Blending Accuracy: %.3f' % score)
end = time.time()
print('Total time is %.3f seconds.' % (end-start))

Get model finished.
Train model finished.
Test model finished.
Blending Accuracy: 0.913
Total time is 4.990 seconds.
